# Initialisation

In [34]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour Windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#import tifffile
#%matplotlib tk #pour Linux

In [35]:
import scipy.ndimage as ndimage
from sklearn.cluster import DBSCAN

# Importer fichier

In [36]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_1um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [37]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [38]:
frame_index = 0
first_frame = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    first_frame += 1
    print(frame_index)




1


# Traitement d'image

In [39]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)
print(img)

[[ 9 10 10 ... 11 11 11]
 [10 10 10 ... 11 11 11]
 [ 9  9 10 ... 11 12 12]
 ...
 [ 9  9 10 ...  8  8  8]
 [ 8  9  9 ...  8  8  8]
 [ 8  9  9 ...  8  9  8]]


# Sélection du point à tracker

In [40]:
def crop(img, x, y, crop_size_x, crop_size_y):
    x_start = int(x - crop_size_x // 2)
    x_end = int(x + crop_size_x // 2)
    y_start = int(y - crop_size_y // 2)
    y_end = int(y + crop_size_y // 2)

    return img[y_start:y_end, x_start:x_end]

In [41]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, origin='lower', cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze = crop_sze_x = crop_sze_y = 100

Please click on the point you want to select.
Selected point: (450.6688311688313, 578.6558441558442)


Debogueur

In [42]:
def visionneur(frame, titre):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title(titre)
    plt.colorbar()  
    plt.show()

# Fit gaussien sur le point sélectionné

In [43]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [44]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [45]:
def localisateur_gaussien(intensity_grid, centre):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))
    
    x_position = result.params['x0'].value + centre[0] - crop_sze_x/2
    y_position = result.params['y0'].value + centre[1] - crop_sze_y/2

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [46]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Passe au prochain frame

In [47]:
# def next_frame(frame_index):
#     frame_index += 1
#     tiff.seek(frame_index)
#     original_image = np.array(tiff)
    
#     while np.sum(original_image) == 0:
#         frame_index += 1
#         tiff.seek(frame_index)
#         original_image = np.array(tiff)

#     return [frame_index, original_image]

# Faire le crop et fit

In [48]:
def particle_tracker(image, x, y):
    image = denoise(image)

    cropped_img = crop(image, x, y, crop_sze_x, crop_sze_y)
    
    #Gérer plus qu'une particule
    cropped_img = np.array(cropped_img)
    max_index = np.argmax(cropped_img)
    max_coords = np.unravel_index(max_index, cropped_img.shape)

    nouveau_x = x - crop_sze_x // 2 + max_coords[1]
    nouveau_y = y - crop_sze_y // 2 + max_coords[0]
    
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x, crop_sze_y)    # Re-crop autour d'une seule particule

    result_fit = localisateur_gaussien(second_crop, [nouveau_x, nouveau_y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [49]:
position_list=[]
sigma_list = []
crop_frames = []
big_frames = []


for i in range(first_frame,actual_frames):
    tiff.seek(i)
    if np.sum(np.array(tiff))==0:
        position_list.append(np.array([np.nan,np.nan]))
        sigma_list.append(np.array([np.nan,np.nan]))
        print("hello")
    else:
        data = particle_tracker(img, x, y)
        position_list.append(data[2])
        x,y = position_list[i-first_frame]
        sigma_list.append(data[3])
        tiff.seek(i)
        img = np.array(tiff)
        big_frames.append(img)
        crop_frames.append(data[1])
    print(position_list)


[(443.35873701478187, 569.6902817526192)]
[(443.35873701478187, 569.6902817526192), (443.11142077118967, 569.8299368270187)]
[(443.35873701478187, 569.6902817526192), (443.11142077118967, 569.8299368270187), (445.7648143014223, 575.9788937257979)]
[(443.35873701478187, 569.6902817526192), (443.11142077118967, 569.8299368270187), (445.7648143014223, 575.9788937257979), (449.7373912577528, 583.1721516675562)]
[(443.35873701478187, 569.6902817526192), (443.11142077118967, 569.8299368270187), (445.7648143014223, 575.9788937257979), (449.7373912577528, 583.1721516675562), (447.9894693020292, 586.5803790929161)]
[(443.35873701478187, 569.6902817526192), (443.11142077118967, 569.8299368270187), (445.7648143014223, 575.9788937257979), (449.7373912577528, 583.1721516675562), (447.9894693020292, 586.5803790929161), (450.4591172651719, 593.6022450336577)]
[(443.35873701478187, 569.6902817526192), (443.11142077118967, 569.8299368270187), (445.7648143014223, 575.9788937257979), (449.7373912577528, 

In [50]:
def calculate_msd_with_uncertainty(positions, delta_x, delta_y):
    """
    Calcule les MSD avec propagation des incertitudes analytiques.
    """
    msd = []
    uncertainties = []
    for d in range(1, len(positions)):
        diff_pairs = positions[d:] - positions[:-d]
        dx = delta_x[d:] + delta_x[:-d]
        dy = delta_y[d:] + delta_y[:-d]
        term_x = 2 * (diff_pairs[:, 0]**2) * (dx**2)
        term_y = 2 * (diff_pairs[:, 1]**2) * (dy**2)

        distances_squared = np.sum(diff_pairs**2, axis=1)
        msd.append(np.mean(distances_squared))  
        
        # Propagation des incertitudes
        total_uncertainty = np.mean(term_x + term_y)
        uncertainties.append(np.sqrt(total_uncertainty))
    
    return np.array(msd), np.array(uncertainties)

# Résultats

In [51]:
sigma_list

[(3.2158228875840003, 4.357856043023636),
 (2.7544597732271177, 4.081611236011488),
 (3.0908964499253893, 4.801012482909487),
 (2.9332089448589276, 4.5455436174720845),
 (2.6224537463411965, 4.738954495248518),
 (2.8679255650694864, 4.763459138895714),
 (2.5780074962388597, 4.043083031471571),
 (2.4847628590590634, 4.174153341717407),
 (2.987051866343308, 4.142380684586837),
 (2.7446358709347405, 4.279547805535049),
 (3.116687442281661, 4.7181997892276515),
 (2.8897746691344186, 4.4302073035412635),
 (2.952542518210483, 4.444030843145389),
 (3.526153714468602, 4.71475032836939),
 (3.283197533721351, 5.1913559256050155),
 (2.900003128882834, 4.739106694798207),
 (2.9762212244063893, 4.308327458557454),
 (2.96451700428865, 4.569906510703644),
 (2.8243959905806952, 4.471440697348459),
 (2.781850432236151, 4.9633802977983255),
 (3.195655306938613, 4.751087426745401),
 (2.764840305475311, 4.5099856594733305),
 (2.9956293957527174, 4.919681004438215),
 (3.154353144720592, 4.510944912187926),

In [52]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Inverser l'axe Y
plt.gca().invert_yaxis()

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

# Afficher le graphique
plt.show()

## anim toutes les particules

In [53]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], origin='lower', cmap='gray', animated=True)


# Update function
def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
plt.show()

## anim crop

In [54]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], origin='lower', cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()